In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import spectools.lpsd as lpsd
import spectools.aux as aux
from pytdi.dsp import timeshift
from multiprocessing import Pool
import scipy.signal.windows as win
from tqdm import tqdm

In [ ]:
# Noise generation
N = int(1e6) # samples
fs = 1e6 # Hertz
T = N/fs # seconds
delay = 1e-4 # seconds
print(f"Total time = {T} seconds")
print(f"Time delay = {delay} seconds")

y0 = np.random.normal(0, 1, N)
y1 = timeshift(y0, delay*fs) + 0.0*np.random.normal(0, 1, N)

n_trunc = int(2*delay*fs)

if n_trunc > 0:
    y0 = y0[n_trunc:-n_trunc]
    y1 = y1[n_trunc:-n_trunc]

print(f"Total length of time series after cropping = {len(y1)}")

coh_NBE = []
coh_dev = []
tf_NBE = []

x_axis = np.linspace(0.01,1.0,20) # delay/T
for x in tqdm(x_axis):
    T = delay/x # record length in seconds
    L = int(T*fs) # record length in integer samples
    tf0 = lpsd.ltf_single_bin([y0,y0], fs=fs, freq=fs/2, L=L, win=win.boxcar, olap=0.95)
    tf1 = lpsd.ltf_single_bin([y0,y1], fs=fs, freq=fs/2, L=L, win=win.boxcar, olap=0.95)
    coh_NBE.append((np.array(tf1.coh) - np.array(tf0.coh))/np.array(tf0.coh))
    coh_dev.append(tf1.coh_dev)
    tf_NBE.append((np.abs(tf1.Hxy) - np.abs(tf0.Hxy))/np.abs(tf0.Hxy))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=150)

# NBE of frequency response estimate (https://doi.org/10.1016/0022-460X(78)90396-6, Equation 23)
ax.plot(x_axis, 1-x_axis-1, c='gray', ls='-', lw=5, alpha=0.5)

# NBE of coherence estimate (https://doi.org/10.1016/0022-460X(78)90396-6, Equation 24)
ax.plot(x_axis, (1-x_axis)**2-1, c='gray', ls='-', lw=5, alpha=0.5)

ax.plot(x_axis, tf_NBE, c='k', ls='-', label=r'$|H_{xy}|$')
ax.plot(x_axis, coh_NBE, c='k', ls='--', label=r'$\gamma_{xy}^2$')

ax.set_xlabel(r'$\tau_0/T$')
ax.set_ylabel(r'Normalized bias error')
ax.set_xlim(0,1)
ax.set_ylim(-1,0)
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, handlelength=2.5)
plt.show()